In [41]:
import pandas as pd
import regex as re
import numpy as np
import math
import enchant
import nltk
import time
from nltk.stem import PorterStemmer
import pickle
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import ngrams

In [106]:
class UGC_final:
    
    def __init__(self,df,db):
        
        self.x = df
        self.db = db
        self.sym_twograms = pickle.load(open('symptoms_twograms.p','rb'))
        self.sym_threegrams = pickle.load(open('symptoms_threegrams.p','rb'))
        self.df_sym = pd.read_csv('~/Downloads/symptoms.csv')
        self.diseases = pd.read_csv('diseases.csv')
        self.user = pd.read_csv('user_features.csv')
        
    def score(self):
        
        feat=self.user.drop('Unnamed: 0',1)
        score = []
        for i in range(len(feat)):
            
    
            if (feat.total_answers[i]>1) or (feat.free_text_answers[i]>0):
                score.append(1)
            else:
                score.append(0)
        feat['score']=score
        abc = feat.drop(feat.columns[1:9],1)
        return (abc)
    
    def pre_processing(self):
        
        x=self.x.sort_values(['visitor_id','id'],ascending=[True,True]).reset_index(drop=True)
        y=x.drop_duplicates('id','first').reset_index(drop=True)
        visitorAggregated = pd.DataFrame(y.groupby("visitor_id")["id"].count()).reset_index()
        visitorSorted = visitorAggregated.sort_values(['id'],ascending=[False]).reset_index(drop=True)
        t=pd.merge(y,visitorSorted, on='visitor_id', how='inner')
        
        z=pd.merge(self.score(),t,how='inner',on='visitor_id')
        return (z)
    
    def free_text(self,txt):
    #print (txt)
        self.df_sym.symptoms = self.df_sym.symptoms.apply(lambda x: re.sub(r'[^a-zA-Z]',' ',x.lower()))
        symptoms = self.df_sym.symptoms.unique()
        words = word_tokenize(txt)
        lemmatiser = WordNetLemmatizer()
        text=''

        for w in words:
            text=text+lemmatiser.lemmatize(w)+' '

        #print (text)

        line=' '.join(text.lower().split())
        line=line.strip().split(' ')
        ar=[]
        two_grams=[]

        three_grams=[]
                        #text_1gram = ngrams((df.text)[j].split(),1)
        text_2grams = ngrams(txt.split(), 2)
        text_3grams = ngrams(txt.split(), 3)
        for grams in text_2grams:


            two_grams.append(grams)
            if grams in self.sym_twograms:
                #print (grams)
                s=''
                for word in list(grams):
                    s=s+word+' '
                    #print (s)
                ar.append(s.rstrip())

        for grams in text_3grams:
            three_grams.append(grams)
            if grams in self.sym_threegrams:
                s=''
                for word in list(grams):
                    s=s+word+' '
                    #print (s)    
                                #print (grams)
                ar.append(s.rstrip())


        for word in line:
            #print (word)

                            #text=text+word+' '
                            #ar = []
            if word in symptoms:
                #print (word)

                ar.append(word)

            if word=='no':
                #print (word)
                ar.append(word)

        return (list(set(ar)))
    
    def return_scoreDict(self,i,db,q_id):
        
        de = dict()

        df = db[(db.entity_id==i)].reset_index(drop=True)
        df.text = df.text.astype(str)
        for j in range(len(df)):
            m,y,n=0,0,0
            #print (df.text[j])
            if  str(df.text[j]).startswith('{'):
                #print (df.text[j])
                a=(df.text[j].split('"value":'))[0]
                a= a.split('"id":')[1]
                if '(' in a:
                    a = a.split('(')[0]
                a2 = a.split(',')

                for k in range(len(a2)):

                    b= re.sub(r'[{}""]','',a2[k])

                    if b.strip() in de:

                        #print (b,y)
                        y=de[b.strip()]+df.score[j]
                        #print (b,y)

                        de[b.strip()]= y
                        y=0

                    else:
                        #print (b,y)
                        y+=df.score[j]
                        #print (b,y)

                        de[b.strip()]=y
                        y=0

            elif (df.text[j].isdigit() is True): #and (diseases[diseases.id==int(df.text[j])].empty is False):

                #dis = diseases[diseases.id==int(df.text[j])].reset_index(drop=True)

                c = df.text[j]
                print (c)
                if c.strip() in de:

                    n= de[c.strip()]+df.score[j]
                    de[c.strip()]=n
                    n=0
                else:

                    n+=df.score[j]
                    de[c.strip()]= n
                    n=0

            else:
                text = self.free_text(df.text[j])
                #print (text)

                for word in text:

                    if word in de:

                        m=de[word]+df.score[j]
                        de[word]=m
                        m=0
                    else:
                        m+=df.score[j]
                        de[word]=m
                        m=0


        di={k: v for k, v in de.items() if k!=''}

        return ([i, q_id, di])
    
    def processing(self):
        
        z=self.pre_processing()
        db = self.db
        #print (z.head(5))
        for i in list(set(z.q_id)):
            print (i)
            df_qid=z[z.q_id==i].reset_index(drop=True)
            #print (df_qid.head(5))
            
            if (i==2):
                
                print ('two')
                for j in list(set(df_qid.entity_id)):
                    e,m,cn,n,xxx = 0,0,0,0,0
                    df = df_qid[(df_qid.entity_id==j)].reset_index(drop=True)


                    for k in range(len(df)):
                        #xxx+=float(df.score[j])

                        if df.text[k]=='Effective':
                            e+=df.score[k]

                        if df.text[k]=='Moderatively effective':
                            m+=df.score[k]
                        if df.text[k]=="Can't say":
                            cn+=df.score[k]
                        if df.text[k]=="Not effective":
                            n+=df.score[k]

                    de={'Effective':e,'Moderatively effective':m,'Cant say':cn,'Not effective':n}
                    db=pd.DataFrame(np.array([[j,2,de]]), columns=['entity_id','q_id','distribution']).append(db, ignore_index=True)
            
            if (i==3):
                print ('three')
                
                for j in list(set(df_qid.entity_id)):
                    
                    e,m = 0,0
                    df = df_qid[(df_qid.entity_id==j)].reset_index(drop=True)


                    for k in range(len(df)):

                        if  df.text[k]=='Yes':
                            e+=df.score[k]
                        else:
                            m+=df.score[k]

                    de={'Yes':e,'No':m}
                    db=pd.DataFrame(np.array([[j,3,de]]), columns=['entity_id','q_id','distribution']).append(db, ignore_index=True)
            
            
            if (i!=2) and (i!=3):
                print ('four')
                
                
                for j in list(set(df_qid.entity_id)):
                    xyx=(self.return_scoreDict(j,df_qid,i))
                    db=pd.DataFrame(np.array([xyx]), columns=['entity_id', 'q_id', 'distribution']).append(db, ignore_index=True)
                    
            
                
        return (db)        
                
            
            
    
    

In [35]:
x = pd.read_csv('base_pharmacy_answers.csv')
db = pd.DataFrame(data=None)

In [108]:
a=UGC_final(x,db)

In [109]:
u=a.processing()

2
two
3
three
4
four
12


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: DeprecationWarning: generator 'ngrams' raised StopIteration


00
1
20
1
12
5
1
11
25181525
12
222222
86
1
555
425
000
1
0
200
45
1
1
1
2249464
111
123
50
25
5
22
12
2
0
00
1
22
1
15
6
four
39
four
409
409
408
191
409
408
91
91
91
91
91
248
157
8
8
323
8
8
323
8
323
323
323
323
323
323
323
323
323
8
43
45
8
105
105
105
105
105
105
105
105
105
105
105
59
8
8
8
471
8
158
49
49
50
51
49
68
68
248
248
8
248
248
248
248
8
8
106
105
105
105
105
105
105
576
233
398
8
89
71
8
149
77
77
77
105
471
269
8
8
471
8
42
8
77
119
20
227
109
233
591
158
8
405
43
181
107
91
91
91
91
91
8
8
8
91
91
91
8
8
91
8
91
91
91
261
261
261
198
192
192
8
80
591
590
192
590
578
83
248
8
248
248
68
457
68
68
406
168
168
49
49
248
248
67
406
406
406
8
188
188
188
261
43
42
42
61
8
61
61
61
61
61
61
61
61
8
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
8
61
61
61
61
8
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
8
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
6

157
8
398
8
8
8
8
8
398
8
68
158
157
156
158
158
157
157
158
156
8
157
158
158
8
158
156
157
158
158
8
156
157
158
8
156
158
8
157
156
158
8
158
158
158
158
157
158
156
158
156
158
156
157
156
8
157
157
156
8
157
157
156
156
156
157
156
8
156
156
157
157
156
156
269
269
269
457
269
158
157
157
156
156
156
156
156
156
156
156
156
156
156
156
156
156
158
157
157
158
8
157
157
158
157
156
156
157
157
248
158
157
156
156
156
191
156
157
156
328
327
328
328
328
328
156
156
157
157
156
157
156
156
156
8
48
48
48
199
8
248
251
251
177
177
242
248
298
157
590
138
138
138
138
138
138
138
138
138
191
191
192
59
59
248
248
398
68
68
68
8
210
210
8
59
8
8
8
59
59
59
59
59
8
59
59
8
8
59
59
8
59
59
59
59
59
59
59
8
59
59
59
59
59
59
8
8
8
59
59
8
8
59
59
59
59
59
59
59
59
59
59
59
59
8
59
59
59
59
59
59
8
59
8
59
59
59
8
59
59
59
59
8
59
59
59
59
8
59
59
59
59
8
59
59
59
587
59
8
59
8
8
59
8
59
59
59
59
59
59
59
59
59
8
8
8
248
59
59
109
98
398
8
248
406
168
168
8
59
59
59
59
8
59
59
59
59
59
59
59

277
148
42
42
42
101
8
89
89
89
89
89
89
89
89
8
8
248
242
8
409
407
408
408
10
264
264
264
8
8
264
264
264
264
8
20
113
113
113
113
113
113
114
8
8
114
113
113
114
8
113
114
113
113
114
114
113
114
113
113
114
210
210
210
68
68
68
68
68
68
68
93
78
89
89
134
134
134
134
134
134
89
89
89
89
89
89
89
89
89
45
89
89
89
89
89
89
89
89
89
68
114
114
20
116
20
20
20
8
8
8
8
179
590
159
327
327
327
327
327
327
256
256
256
256
256
8
157
173
173
157
8
68
363
8
363
157
156
156
264
264
264
264
264
264
264
264
264
264
264
8
264
264
264
264
314
591
590
590
590
260
260
591
590
590
192
590
591
481
591
591
8
105
299
8
299
8
299
113
8
8
114
113
8
113
8
113
8
113
8
113
113
113
260
261
261
261
261
191
83
248
68
68
109
68
68
105
106
105
105
105
105
105
148
148
105
198
591
8
67
269
269
198
591
198
198
591
591
8
323
591
213
198
248
191
192
191
196
191
591
591
212
212
248
591
8
213
213
213
212
20
116
172
116
61
198
213
591
198
8
248
8
8
248
591
591
591
8
198
591
198
375
8
198
212
591
105
591
212
261
591
212

...

2
4
3
3
1
1
1
1
3
4
2
3
3
5
4
1
3
4
2
5
5
2
3
3
4
1
3
1
2
4
5
4
3
5
1
3
4
1
4
4
2
5
2
2
3
5
4
4
3
5
1
5
3
1
2
5
3
3
4
2
3
5
3
1
4
3
5
3
5
1
3
3
2
2
3
3
5
5
3
2
4
3
3
3
3
3
5
4
4
3
4
3
5
4
5
1
2
4
4
1
2
4
5
5
1
4
3
1
3
1
4
3
3
5
5
4
5
2
2
2
5
4
4
1
5
1
1
5
4
2
4
1
4
2
2
2
5
3
3
1
4
5
2
2
2
3
3
2
2
1
4
5
3
3
2
1
4
4
2
5
1
4
4
3
4
4
1
4
2
4
2
2
1
2
3
1
5
2
3
1
3
4
3
1
4
1
3
1
2
1
3
3
3
2
2
4
5
5
1
4
4
4
4
4
4
3
2
4
3
3
5
3
4
3
3
5
4
3
4
3
3
1
1
3
3
3
4
5
2
4
3
3
5
3
2
5
2
4
4
2
4
3
4
3
2
3
3
2
3
4
3
2
4
4
1
3
4
2
5
5
2
4
5
1
4
4
3
4
3
1
3
3
4
4
2
1
3
4
4
1
2
2
5
2
4
2
2
4
5
1
5
4
4
4
2
3
3
5
2
2
5
3
5
4
3
5
1
4
5
2
2
2
1
5
5
4
1
4
1
3
3
5
5
2
1
3
2
1
4
1
4
4
5
4
3
5
3
4
2
2
4
3
3
4
2
1
3
4
4
2
2
4
4
4
3
3
4
2
4
4
4
5
4
2
2
5
2
1
4
1
1
5
2
5
5
3
2
5
4
2
2
3
5
5
5
4
2
4
3
3
2
1
2
3
2
2
1
2
1
5
5
3
1
4
1
3
4
3
1
4
3
3
2
3
3
3
4
1
3
4
1
1
3
3
1
4
3
3
3
1
4
4
3
3
3
5
3
3
1
4
1
2
1
3
3
2
2
1
1
3
5
3
1
2
5
3
5
4
1
3
2
2
5
5
3
4
5
3
4
3
2
4
3
4
3
3
4
3
5
1
1
3
3
3
2
2
5
3
1
1
4
3
4
3
4
5
3
4
1
3


3
4
5
7
3
14
3
3
5
4
4
7
4
1
2
4
4
2
7
3
3
2
3
3
3
3
2
2
4
4
4
7
1
14
4
2
4
3
1
1
3
1
1
2
1
7
4
4
2
4
2
3
4
28
5
4
5
28
4
21
5
3
7
3
28
2
7
2
5
4
5
21
5
4
3
3
1
14
4
14
14
4
5
4
3
4
4
1
4
2
4
1
1
4
4
3
1
7
4
4
4
2
2
3
3
3
7
7
1
3
3
2
21
1
3
3
5
1
7
3
5
2
2
2
2
1
1
3
2
2
5
5
4
2
28
1
14
28
3
7
2
2
4
5
4
7
4
4
7
3
4
7
3
2
2
4
5
5
28
5
2
3
2
14
3
7
4
2
14
5
5
14
7
2
7
7
7
2
3
5
3
5
3
5
4
14
3
3
5
2
1
4
3
3
3
7
4
5
14
14
4
1
14
2
5
3
5
5
5
5
5
28
7
4
5
14
21
3
2
3
21
21
4
4
7
5
4
2
5
2
3
3
14
4
4
28
3
14
1
1
3
4
3
4
3
7
4
4
5
28
7
7
3
3
3
28
5
3
7
1
1
4
3
5
3
4
2
4
2
1
3
2
2
4
1
4
2
7
21
28
2
3
4
28
1
3
28
2
1
4
1
2
4
3
4
2
2
2
3
3
1
3
7
5
2
4
4
1
4
14
3
5
3
5
1
3
5
5
4
3
5
28
5
7
4
5
7
1
1
2
1
5
1
7
3
3
3
2
1
5
4
5
3
7
1
3
3
14
1
3
1
3
1
3
7
1
1
3
1
3
21
4
5
2
2
5
2
5
3
5
5
1
28
5
2
2
3
4
28
1
4
7
5
4
1
2
14
3
2
3
2
2
4
7
3
3
3
5
4
21
2
4
28
28
2
1
3
4
3
1
7
3
3
3
7
14
3
7
28
3
3
1
3
21
1
2
4
2
1
1
28
1
7
3
4
3
4
5
4
1
3
4
5
3
2
4
4
4
3
5
3
7
1
3
5
7
3
2
4
2
7
3
5
3
3
28
7
7
3
1
1
3
2
2
3

4
14
5
14
2
28
5
4
7
1
4
4
5
3
4
1
3
1
7
7
7
14
5
28
1
2
1
3
7
4
3
5
7
28
3
2
4
28
1
1
3
4
4
2
2
5
5
4
2
2
1
7
5
3
3
7
1
3
1
1
5
1
2
1
2
2
4
1
1
1
5
3
5
3
2
2
5
1
1
5
1
1
1
1
1
1
1
1
1
1
3
5
5
14
2
3
7
1
3
28
3
21
28
7
5
2
2
4
1
1
7
3
4
1
4
5
3
3
2
2
5
4
1
1
1
4
3
1
5
2
5
4
5
14
3
4
14
4
7
3
1
4
4
1
2
2
21
3
3
3
28
7
2
1
1
2
3
2
14
2
1
3
28
14
4
1
3
2
3
3
5
5
4
7
3
2
5
7
4
3
4
4
3
4
1
4
28
3
3
2
28
14
5
5
3
4
4
14
4
3
5
5
5
2
14
2
4
5
3
7
4
5
5
3
5
7
1
4
21
2
1
5
4
2
5
1
1
5
3
2
5
4
1
5
28
5
3
4
5
3
4
3
1
14
5
3
4
28
2
5
1
3
7
7
7
1
1
7
3
3
1
3
3
3
5
4
2
3
1
4
3
14
28
28
3
5
7
1
3
1
2
2
4
1
1
3
28
3
2
28
2
5
7
7
2
4
28
7
5
2
4
14
3
3
14
3
2
21
14
2
2
3
1
1
1
3
4
21
3
14
7
1
7
1
2
2
2
2
1
21
4
1
2
4
5
1
14
4
28
21
28
21
4
14
5
5
1
4
4
5
1
3
5
4
3
4
5
2
5
5
1
5
3
3
3
4
3
3
4
4
3
2
2
7
1
4
7
5
28
4
4
2
1
3
5
5
4
3
2
3
2
7
5
14
4
3
1
4
1
4
5
1
4
5
1
7
5
28
3
3
2
2
3
3
3
4
1
3
4
4
7
1
2
2
4
3
1
7
1
1
1
3
7
1
3
7
3
14
1
1
5
5
3
3
7
2
4
3
1
2
3
4
2
1
2
28
28
1
4
7
5
4
1
4
7
1
5
2
5
7
7
7
5
4


4
4
3
14
5
2
2
4
1
2
3
1
1
2
2
4
5
4
1
1
2
3
2
4
3
5
3
14
7
1
14
2
3
2
28
14
3
21
5
3
4
1
1
4
1
5
1
3
3
5
14
7
2
5
3
1
4
4
2
2
4
2
2
7
3
3
1
1
1
4
3
4
14
5
5
3
2
7
2
1
2
1
7
1
28
3
4
3
2
3
4
2
5
5
7
5
7
2
3
2
2
4
5
4
5
5
4
21
3
4
3
14
5
3
1
1
1
1
1
3
1
1
4
3
3
3
5
4
2
4
7
4
7
1
1
7
1
3
4
28
28
3
1
5
1
1
5
14
3
4
5
7
7
5
2
7
28
7
3
4
5
1
3
3
3
3
7
1
3
7
2
2
1
1
5
1
5
5
5
4
4
5
4
5
5
5
3
5
5
2
4
3
5
7
1
3
4
4
5
3
5
7
7
1
1
1
7
4
4
1
3
5
4
2
7
3
5
7
1
4
28
4
1
28
2
3
1
4
4
1
2
1
1
1
1
7
4
4
4
1
5
3
5
3
2
5
1
2
2
2
4
3
1
4
4
7
1
4
3
1
4
5
1
3
4
3
1
14
14
2
2
1
1
5
3
1
21
2
3
7
28
2
2
3
3
4
7
7
2
28
1
3
7
4
4
3
1
1
2
4
3
4
3
4
1
3
1
5
2
4
1
1
3
7
2
5
4
5
5
5
1
3
7
4
28
4
1
1
3
4
3
3
4
1
14
3
4
4
1
5
28
14
5
1
1
7
2
1
2
5
3
4
2
1
5
2
2
28
5
2
3
28
1
1
2
2
1
7
3
28
7
4
21
3
5
1
2
7
5
4
1
1
5
1
2
2
7
2
2
3
2
3
4
2
4
2
14
7
3
1
5
21
21
1
2
4
5
5
7
2
2
3
3
1
3
4
1
3
3
1
4
21
7
4
7
7
3
4
5
3
7
3
7
2
4
1
1
3
1
7
1
28
7
5
2
1
4
28
2
5
1
4
4
4
2
5
3
4
4
1
2
3
3
4
1
1
3
1
3
1
1
7
4
1
5
2
3
5
1
1
4
1


3
1
3
2
7
3
7
21
7
7
5
3
7
4
5
3
4
28
14
7
2
1
7
3
14
2
3
5
4
2
5
28
1
3
3
4
1
28
2
28
1
5
1
1
1
4
1
5
1
4
1
1
3
2
4
1
28
14
4
14
28
7
4
3
4
3
28
3
1
21
1
3
7
4
2
7
4
1
7
7
5
28
7
28
3
5
7
7
21
3
3
4
5
1
2
4
3
2
14
2
5
28
1
1
14
1
3
7
1
28
3
14
4
3
5
7
4
4
7
3
2
2
2
2
2
7
1
3
1
2
4
28
4
2
2
2
4
4
7
3
3
1
3
5
1
7
1
1
3
4
5
5
2
5
4
1
1
3
3
14
3
2
5
2
2
2
1
3
1
14
7
4
4
3
1
14
3
7
1
1
3
1
3
7
3
3
4
2
4
4
2
28
2
28
3
28
4
14
3
7
3
2
2
1
3
5
3
4
3
5
1
7
5
28
4
3
3
4
3
4
1
5
3
4
1
7
2
5
28
3
3
5
2
3
1
4
5
4
2
3
2
28
5
21
28
5
3
5
4
2
2
2
4
1
7
2
4
5
7
1
3
5
3
2
28
7
2
4
7
3
5
2
7
1
5
1
2
3
4
3
14
28
5
4
4
14
3
3
3
1
5
1
21
3
1
2
3
14
3
1
5
4
14
3
3
3
1
3
1
3
3
1
5
4
5
3
1
3
1
3
21
2
1
5
3
3
4
2
1
2
2
3
7
2
7
2
3
7
5
1
4
2
1
2
5
3
1
3
7
5
28
7
3
3
7
2
3
4
3
3
1
4
3
4
21
3
3
5
4
3
3
2
4
5
1
1
7
3
7
7
1
3
1
5
1
28
28
3
1
28
3
1
7
28
5
28
2
28
3
7
1
28
3
4
28
1
7
7
2
14
4
3
3
3
5
3
3
28
14
3
14
7
2
5
3
5
2
5
1
4
5
4
4
4
3
7
3
1
5
14
7
3
28
5
2
28
5
28
2
14
28
4
28
7
4
3
7
14
4
4
28
2
4
3
3
5
21


In [113]:
xx=u.sort_values(by=['entity_id','q_id'],ascending=[True,True]).reset_index(drop=True)

In [117]:
xx.to_csv('UGC_distribution.csv')

In [37]:
pd.set_option('display.max_rows', len(x))

In [41]:
x[(x.entity_id==5135) & (x.q_id==41)].text[70235]

'{"id": "2852,1208,1186,1345", "value": "Vomiting,Loss of appetite,Sleepiness,Hair loss"}'

In [14]:
diseases = pd.read_csv('diseases.csv')

In [43]:
xx[xx.entity_id==5135].distribution[193]

"{'breathlessness': 1, 'tremor': 1, 'no': 1, 'weakness': 1}"

In [31]:
diseases[diseases.name=='Hair loss']

,id,name
754,77,Hair loss
880,77,Hair loss
1006,77,Hair loss
1132,77,Hair loss
1258,77,Hair loss
1510,77,Hair loss
1636,77,Hair loss
1762,77,Hair loss
1888,77,Hair loss
2014,77,Hair loss


In [1]:
import pandas as pd
xx = pd.read_csv('UGC_distribution.csv')

In [45]:
import pandas as pd
df = pd.read_csv('ugh_questions.csv')

In [28]:
import pickle
skus = pickle.load(open('entity_id2q_41.p','rb'))

In [30]:
skus[5135]

{'Hair loss': 2, 'Loss of appetite': 1, 'Sleepiness': 2, 'Vomiting': 1}

In [47]:
xx[xx.entity_id==5135].distribution[193]

"{'breathlessness': 1, 'tremor': 1, 'no': 1, 'weakness': 1}"

In [32]:
xx[xx.id==5135]

AttributeError: 'DataFrame' object has no attribute 'id'

In [46]:
df.sort_values(by='id',ascending=[True]).test[2]

'Any specific experiences or tips regarding side-effects of this medicine?'